In [0]:
# it's defau;t spark session in Jupiter Notebook
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
dbutils.fs.put("/scenarios/duplicates.csv", """id,name,loc,updated_date
1,ravi,bangalore,2021-01-01
1,ravi,chennai,2022-02-02
1,ravi,Hyderabad,2022-06-10
2,Raj,bangalore,2021-01-01
2,Raj,chennai,2022-02-02
3,Raj,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
5,Mahesh,chennai,2022-02-02
4,Prasad,Hyderabad,2022-06-10
""")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1300737379233180>:1
----> 1 dbutils.fs.put("/scenarios/duplicates.csv","""id,name,loc,updated_date
      2 1,ravi,bangalore,2021-01-01
      3 1,ravi,chennai,2022-02-02
      4 1,ravi,Hyderabad,2022-06-10
      5 2,Raj,bangalore,2021-01-01
      6 2,Raj,chennai,2022-02-02
      7 3,Raj,Hyderabad,2022-06-10
      8 4,Prasad,bangalore,2021-01-01
      9 5,Mahesh,chennai,2022-02-02
     10 4,Prasad,Hyderabad,2022-06-10
     11 """)

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o1794.put.
: org.apache.hadoop.fs.FileAlreadyExistsException: s3a://databricks-prod-storage-oregon/devtierprod1/10552

In [0]:
%fs
ls /scenarios/

path,name,size,modificationTime
dbfs:/scenarios/duplicates.csv,duplicates.csv,274,1701845455000


In [0]:
dublicate_df = spark.read.csv("/scenarios/duplicates.csv", inferSchema=True, header=True)
dublicate_df.orderBy(col("id"), col("updated_date").desc()).display()

id,name,loc,updated_date
1,ravi,Hyderabad,2022-06-10
1,ravi,chennai,2022-02-02
1,ravi,bangalore,2021-01-01
2,Raj,chennai,2022-02-02
2,Raj,bangalore,2021-01-01
3,Raj,Hyderabad,2022-06-10
4,Prasad,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
5,Mahesh,chennai,2022-02-02


In [0]:
# here we get same output
# that means we are not remove Duplicates
dublicate_df.dropDuplicates().display()
dublicate_df.distinct().display()

id,name,loc,updated_date
2,Raj,bangalore,2021-01-01
1,ravi,bangalore,2021-01-01
1,ravi,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
4,Prasad,Hyderabad,2022-06-10
3,Raj,Hyderabad,2022-06-10
2,Raj,chennai,2022-02-02
1,ravi,chennai,2022-02-02
5,Mahesh,chennai,2022-02-02


id,name,loc,updated_date
2,Raj,bangalore,2021-01-01
1,ravi,bangalore,2021-01-01
1,ravi,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
4,Prasad,Hyderabad,2022-06-10
3,Raj,Hyderabad,2022-06-10
2,Raj,chennai,2022-02-02
1,ravi,chennai,2022-02-02
5,Mahesh,chennai,2022-02-02


In [0]:
# to remove duplicates we use column name

from pyspark.sql.functions import *

dublicate_df.orderBy(col("updated_date").desc()).dropDuplicates(["id"]).display()

id,name,loc,updated_date
1,ravi,Hyderabad,2022-06-10
2,Raj,chennai,2022-02-02
3,Raj,Hyderabad,2022-06-10
4,Prasad,Hyderabad,2022-06-10
5,Mahesh,chennai,2022-02-02


# we can do this using window function (row_number())

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
df = dublicate_df.withColumn("rowid",row_number().over(Window.partitionBy("id").orderBy(col("updated_date").desc())))

In [0]:
df.filter("rowid=1").display() # here we get all unique data according to condition

id,name,loc,updated_date,rowid
1,ravi,Hyderabad,2022-06-10,1
2,Raj,chennai,2022-02-02,1
3,Raj,Hyderabad,2022-06-10,1
4,Prasad,Hyderabad,2022-06-10,1
5,Mahesh,chennai,2022-02-02,1


In [0]:
df.filter("rowid>1").display() # here we get data that contain duplicates

id,name,loc,updated_date,rowid
1,ravi,chennai,2022-02-02,2
1,ravi,bangalore,2021-01-01,3
2,Raj,bangalore,2021-01-01,2
4,Prasad,bangalore,2021-01-01,2
